In [1]:
# import time to count running time
import time
startTime = time.time()
# import pm2.5 value
weather_data = sc.textFile("hdfs://master:9000/user/datasets/pm25/pm25Taiwan.csv")

In [2]:
# split by comma
weather_data_rdd = weather_data.map(lambda line : line.split(","))
##############################print###############################
lis = weather_data_rdd.take(3)
for x in lis:
    for i in range(len(x)):
        print x[i],
    print
##################################################################

日期 測站 測項 00 01 02 03 04 05 06 07 08 09 10 11 12 13 14 15 16 17 18 19 20 21 22 23
2015/01/01 龍潭 AMB_TEMP 14 14 14 13 13 13 12 12 13 14 14 14 14 14 13 13 12 11 11 11 11 11 11 11
2015/01/01 龍潭 CO 0.69 0.72 0.69 0.64 0.54 0.47 0.45 0.48 0.51 0.54 0.54 0.5 0.47 0.38 0.36 0.35 0.34 0.37 0.34 0.29 0.26 0.22 0.19 0.18


In [3]:
# remove not pm2.5, the value isn't decimal
import math
def remove_row_with_noise(x):
    for i in range(3, len(x)):
        if not x[i].isdecimal():
            return False
    return True

pm25_data = weather_data_rdd.filter(lambda x: x[2] == u"PM2.5")\
                     .filter(remove_row_with_noise)
    
# create every station name list (it will waste a lot of time, because of collect)
stationNames = pm25_data.map(lambda x:x[1]).distinct().collect()
##############################print###############################
lis = pm25_data.take(3)
for x in lis:
    for i in range(len(x)):
        print x[i],
    print

for x in stationNames:
    print x,
print
##################################################################

2015/01/01 龍潭 PM2.5 46 71 76 74 65 62 56 50 52 56 54 47 40 36 37 27 30 25 26 24 18 16 11 14
2015/01/02 龍潭 PM2.5 15 12 9 14 17 20 18 22 21 23 18 25 24 27 18 23 18 19 18 21 23 18 19 19
2015/01/03 龍潭 PM2.5 14 14 9 13 12 15 9 11 14 27 27 24 16 27 38 39 35 32 38 36 34 36 36 39
龍潭 安南 汐止 善化 士林 潮州 新莊 三義 竹東 永和 忠明 馬公 埔里 中山 鳳山 頭份 板橋 小港 彰化 萬里 苗栗 朴子 西屯 南投 橋頭 竹山 屏東 馬祖 林口 斗六 嘉義 前金 沙鹿 菜寮 三重 基隆 古亭 新店 松山 前鎮 新竹 臺南 臺西 林園 桃園 金門 臺東 大園 觀音 楠梓 平鎮 陽明 大同 新港 花蓮 麥寮 崙背 復興 中壢 新營 土城 淡水 大寮 湖口 關山 萬華 仁武 美濃 大里 豐原 左營 線西 恆春 冬山 宜蘭 二林


In [4]:
# create the data style that we want
def Generated_Measurement(x):
    date = x[0]
    location = x[1]
    measure = x[2]
    measurements_of_a_day = []
    for i in range(3, len(x)):
        measurements_of_a_day.append((date, location, measure, "hr"+str(i-3), x[i]))
    return measurements_of_a_day

pm25_Measurement = pm25_data.flatMap(Generated_Measurement)
##############################print###############################
lis = pm25_Measurement.take(3)
for x in lis:
    for i in range(len(x)):
        print x[i],
    print
##################################################################

2015/01/01 龍潭 PM2.5 hr0 46
2015/01/01 龍潭 PM2.5 hr1 71
2015/01/01 龍潭 PM2.5 hr2 76


In [5]:
# arrange data, put date and hour together
pm25_KeyValue = pm25_Measurement.map(lambda x: ( (x[0], x[3]), (x[1], x[4] ) ) )
##############################print###############################
lis = pm25_KeyValue.take(3)
for x in lis:
    for i in range(len(x)):
        print x[i],
        print
##################################################################

(u'2015/01/01', 'hr0')
(u'\u9f8d\u6f6d', u'46')
(u'2015/01/01', 'hr1')
(u'\u9f8d\u6f6d', u'71')
(u'2015/01/01', 'hr2')
(u'\u9f8d\u6f6d', u'76')


In [6]:
# calculate correlation with one of other station rdd and Dali
from pyspark.mllib.stat import Statistics
Dali_pm25 = pm25_KeyValue.filter(lambda x: x[1][0] == u"大里")
corr_list_with_Dali = []
for i in range(len(stationNames)):
    if stationNames[i] != u"大里":
        One_station = pm25_KeyValue.filter(lambda x: x[1][0] == stationNames[i])
        join_with_Dali = Dali_pm25.join(One_station)
        seriesDali = join_with_Dali.map(lambda x:float(x[1][0][1]))
        seriesX = join_with_Dali.map(lambda x:float(x[1][1][1]))
        tempCorr = str(Statistics.corr(seriesDali, seriesX, method="pearson"))
        corr_list_with_Dali.append([stationNames[i]] + [tempCorr])


In [7]:
# create correlation dataframe
corr_mat_rdd = sc.parallelize(corr_list_with_Dali)
dfColumn = ["Locations","Dali"]
corr_mat_df = corr_mat_rdd.toDF(dfColumn)

In [8]:
# select the top three correlation with Dali
from pyspark.sql.functions import *
ordered_Corr_withDali = corr_mat_df.select("Locations","Dali").orderBy(desc("Dali"))
ordered_Corr_withDali.show(3)

+---------+--------------+
|Locations|          Dali|
+---------+--------------+
|       忠明|0.880213058015|
|       西屯|0.858951681025|
|       南投|0.830540673781|
+---------+--------------+
only showing top 3 rows



In [9]:
# print the total running time
endTime = time.time()
print (endTime-startTime)

314.68484807
